# Data Engineer Challenge 

In [ ]:
from pathlib import Path

from q1_memory import q1_memory
from q1_time import q1_time
from q2_memory import q2_memory
from q2_time import q2_time
from q3_memory import q3_memory
from q3_time import q3_time


In [ ]:
%load_ext memory_profiler

In [ ]:
file_path = (
	Path().cwd().parent.absolute() / "data" / "farmers-protest-tweets-2021-2-4.json"
)

## Problema 1

Se construyó una estrategia basada en la bilioteca `polars`, utilizando un enfoque tabular con un procesamiento muy similar a sentencias `sql`. En otro enfoque, se utilizaron solo bibliotecas estándar para leer el archivo línea por línea, procesar cada una y generar una estructura de diccionario.

### Comparación tiempo de ejecución

In [ ]:
%%timeit
q1_time(file_path)


In [ ]:
%%timeit
q1_memory(file_path)

La diferencia en tiempo de ejecución es mínima (± 0.21 s), teniendo menor tiempo la estrategia inicialmente pensada para optimizar memoria. Esto se puede deber a que el enfoque basado en `polars` contiene agrupaciones/agregaciones y ordenamiento de un conjunto no menor de datos. 

### Comparación de memoria

In [ ]:
%%memit
q1_time(file_path)

In [ ]:
%%memit
q1_memory(file_path)

Los resultados son claros, el enfoque `q1_time` utiliza aproximadamente 2.5 veces la memoria del enfoque `q1_memory`. En la estrategia `q1_time`, a pesar de que se utiliza un método para leer el archivo de forma *Lazy* y poder realizar algunas optimizaciones en uso de memoria, hay varias operaciones costosas como *group_by*  y *sort*.
En la estrategia utilizada en `q1_memory` se lee el archivo línea por línea con una fuente stream (*TextIOWrapper*), sin leer todo el archivo en memoria. Además, se utilizan solo bibliotecas estándar que muy probablemente estén implementadas en *C*. 

## Problema 2

Se construyó una estrategia leyendo el archivo línea por línea y haciendo el conteo en un diccionario. La otra estrategia, transforma el archivo y lo lee completo para hacer el conteo utilizando *Counter*.

### Comparación tiempo ejecución

In [ ]:
%%timeit
q2_time(file_path)

In [ ]:
%%timeit
q2_memory(file_path)

Existe una diferencia mínima (± 0.19 s), teniendo menor tiempo de ejecución la solución que inicialmente estaba pensada en optimizar memoria. Leer todo el archivo y posteriormente transformarlo a una estructura *python* resulta más costoso que hacerlo línea por línea, pero se podría ver compensado con el conteo utilizando *Counter*.

### Comparación uso de memoria

In [ ]:
%%memit
q2_time(file_path)

In [ ]:
%%memit
q2_memory(file_path)

En este experimento el resultado es rotundo, la solución `q2_time` utiliza aproximadamente 10 veces la memoria de `q2_memory`. Leer y transformar el archivo de una sola vez resultaría inviable en muchos escenarios que involucren archivos medianamente grandes. Por lo anterios, no es una solución que escale bien.</br>
Para obtener los mejor de ambos casos, se podría explorar una solución que recorra el archivo línea por línea, agregue los *emoji* a una lista y finalmente aplique *Counter* para realizar el conteo.

## Problema 3

Se construyó una solución basada en `polars` y otra donde se lee el archivo línea por línea para hacer el conteo en un diccionario.

### Comparación tiempo ejecución

In [ ]:
%%timeit
q3_time(file_path)

In [ ]:
%%timeit
q3_memory(file_path)

Existe una diferencia de ± 0.4 s, teniendo menor tiempo de ejecución la estrategia que inicialmente estaba pensada para optimizar la memoria. Esto se puede deber a que en el enfoque que utiliza `polars` en una etapa temprana se tienen columnas de tipo *List[Struct]* que luego se les debe aplicar *explode* y *aplanar*, operaciones que resultan costosas. 

### Comparación uso de memoria

In [ ]:
%%memit
q3_time(file_path)

In [ ]:
%%memit
q3_memory(file_path)

La estrategia `q3_time` utiliza aproximadamente el doble de memoria que `q3_memory`. Esto se debe a que cada línea se procesa de forma individual y secuencial, a diferencia del otro enfoque donde se opera sobre todos los datos en un dataframe.

## Conclusiones

A pesar de que para los 3 problemas los enfoques donde se optimiza la memoria tuvo mejores resultados (marginalmente en tiempo y contundentes en memoria), los enfoques basados en `polars` proporcionan una *API* muy similar a *SQL*, lo que facilita su desarrollo sin comprometer en gran medida el desempeño.
Además, la fuente de datos para los problemas es un archivo *json*, el cual naturalmente resulta más cercano a un diccionario/lista de python que a datos tabulares.
Esto queda en clara evidencia cuando en el dataframe resultan columnas de tipo *nested* que normalmente requieren un procesamiento costoso.

## Validación rápida que las salidas sean consistentes

In [ ]:
for ix, (s1, s2) in enumerate(
    ((q1_time, q1_memory), (q2_time, q2_memory), (q3_time, q3_memory)), 1
):
    if (r1 := s1(file_path)) != (r2 := s2(file_path)):
        print(f"Inconsistencia en Problema {ix}\nOutput 1: {r1}\nOutput 2: {r2}\n")